In [1]:
import httpx
from openai import OpenAI
import os

client = OpenAI(
    http_client=httpx.Client(proxy="<YOUR PROXY>"),  
    api_key="<YOUR_KEY>".encode('ascii', 'ignore').decode('ascii')
    )

In [1]:
import torch, torchvision
from torch import nn
from torchvision import models, transforms, utils
from torch.utils.data import DataLoader
import torch, os
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as datasets
import torch.utils.data as data
import torchvision.transforms as transforms
import torchvision.models as models
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm
import torch.nn.functional as F
import torch.optim as optim
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time
import os

In [2]:
from tqdm import tqdm
import argparse
import json
import os
import pathlib
import re

import time

NUM_SECONDS_TO_SLEEP = 0.5


import openai
from openai import OpenAI

In [6]:
import os
import glob
from pathlib import Path
import PIL
from PIL import Image

In [7]:
import base64
import requests
from openai import OpenAI


import matplotlib.colors as mcolors
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches

def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

In [8]:
with open('<NAME>.json', 'r') as file: #original bboxes coordinates
    orig_maira = json.load(file)

with open('<NAME>.json', 'r') as file: #bboxes coordinates produced with MedRPG on step 1
    medrpg = json.load(file)
    
with open('<NAME>.json', 'r') as file: #bboxes coordinates produced with Maira on step 2
    maira = json.load(file)

with open('<NAME>.json', 'r') as file:  #iou values for every picture produced with MedRPG on step 1
    medrpg_iou = json.load(file)
    
with open('<NAME>.json', 'r') as file: #iou values for every picture produced with Maira on step 1
    maira_iou = json.load(file)

In [3]:
gold_500 = pd.read_csv('gold_500.csv')
body_parts = gold_500.bbox_name.unique()


In [12]:
# confusion matrix
TP = 0
TN = 0
FP = 0
FN = 0
confusion_matrix = {}
#iou collecter
iou_gpt ={}

#collector of the predicted method
workers = {}

#images, that were not classified by chat gpt
outliers = {}

# number of tries
n_tries = 5

In [12]:
from tqdm import tqdm

for part in body_parts:
    
    TP = 0
    TN = 0
    FP = 0
    FN = 0
    confusion_matrix[part] = []
    iou_gpt[part] = []
    workers[part] = []
    
    
    l =  os.listdir(f"without_gt_new/{part}") # link to the folder with images
    for image in tqdm(list(maira[part].keys())):
        
        path = f"without_gt_new/{part}/{image}"
        if (os.path.exists(path)):
            img_try = Image.open(path).convert("RGB").resize((224, 224))


            promt2 = f"You are given the image with two bounding boxes. The first bbox is red, the second bbox is yellow. Please, analize the image and and tell what bbox describes the position of {part} better. Print Maira in case of the red bbox, MedRPG in case of the yellow bbox.  Please, send ONLY one word"

            base64_image_1_224 = encode_image(path)

            response = client.chat.completions.create(
                    model="gpt-4o",
                    messages=[
                    {
                      "role": "user",
                      "content": [
                        {"type": "text", "text": promt2},
                        {
                          "type": "image_url",
                          "image_url": {
                              "url": f"data:image/jpeg;base64,{base64_image_1_224}",
                          },
                        },
                      ],
                    }
                  ],
                  max_tokens=300,
                  )

            response = response.choices[0].message.content
            
            maira_gt_iou = maira_iou[part][image]
            medrpg_gt_iou = medrpg_iou[part][image]
            
            
            # in case chat gpt doesnot provide an answer accoring to the pattern we need to ask chat gpt again
            if response.lower() not in ["maira", "medrpg"]:
                for attempt in range(n_tries):
                    r = client.chat.completions.create(
                    model="gpt-4o",
                    messages=[
                    {
                      "role": "user",
                      "content": [
                        {"type": "text", "text": promt2},
                        {
                          "type": "image_url",
                          "image_url": {
                                  "url": f"data:image/jpeg;base64,{base64_image_1_224}",
                              },
                            },
                          ],
                        }
                      ],
                      max_tokens=300,
                      )
                    r = r.choices[0].message.content
                    if r.lower() in ["maira", "medrpg"]:
                        response = r
                        maira_gt_iou = maira_iou[part][image]
                        medrpg_gt_iou = medrpg_iou[part][image]
                        print("coorected response: ", r)
                        break
                    else:
                        if part not in outliers.keys():
                            #in case chatgpt still did not provide an answer we mark the picture as outlier
                            outliers[part] = [path]
                        else:
                            outliers[part].append(path)
                        maira_gt_iou = 0
                        medrpg_gt_iou = 0
                        print("incorrect response: ",r)





            if response.lower() == "maira":
                    iou_gpt[part].append(maira_gt_iou)
            else:
                if response.lower() == "medrpg":
                    iou_gpt[part].append(medrpg_gt_iou) 
                else:
                    iou_gpt[part].append(0) 
            workers[part].append(response.lower())


            if response.lower() == "maira":
                    if maira_gt_iou>medrpg_gt_iou:
                        TP += 1
                    if maira_gt_iou<=medrpg_gt_iou:   
                        FP +=1
            else :
                if response.lower() == "medrpg":
                    if maira_gt_iou>=medrpg_gt_iou:
                        FN += 1
                    if maira_gt_iou<medrpg_gt_iou: 
                        TN +=1

            confusion_matrix[part] = (TP, FP, TN, FN )

        else:
            pass
            
            

        with open('<CONFUSION MATRIX FILE NAME>.json', 'w') as json_file:
            json.dump(confusion_matrix, json_file, allow_nan=True)
        with open('<IOU FILE NAME>.json', 'w') as json_file:
            json.dump(iou_gpt, json_file, allow_nan=True)
        with open('<WORKERS FILE NAME>.json', 'w') as json_file:
            json.dump(workers, json_file, allow_nan=True)
        with open('<OUTLIERS FILE NAME>.json', 'w') as json_file:
            json.dump(outliers, json_file, allow_nan=True)
  
  

carina


100%|██████████| 16/16 [00:42<00:00,  2.64s/it]


### 